In [1]:
"""

Problem:
Algorithm used for extracting subwords did not feel satisfactory.

Goal:   Reduce the amount of words in the dictionary by decomposing english into the components (affixes) that give meaning to a word
        Target affixes that are most frequently reused and carry meaning consistently
        Final product is a dictionary that will be able to break down any word into its components while maintaining information integrity

Terms
Affix: A word or fragment of a word that carries meaning.
    - Words can be composed entirely of affixes, or out of affixes and a root, or exist only as a root
    - An affix must carry consistent meaning
    - Denoted as a prefix by a underscore on the left or as a suffix by a underscore on the right '_pre' | 'ing_'
Parsing: finding all cases of an affix, counting them and extracting from their root word appropriately, then returning the word to the word list
     OR: finding a subset of words and identifying and counting all affixes before removing the word from the word list

Subgoals:
Create a list of affixes that compose the english language from a list of english words
Retain as much information as possible while transitioning the word list to the affix list
Nested affixes in the affix list should be seperated
The final affix list should be able to compose most words in english
Words should not be broken down into affixes if it destroys the meaning of the word
Words should remain as whole words regardless of length if it cannot be broken down
The algorithm should accomplish its goal with very little manual intervention

Procedures
1) Isolate commonly used words that are affixes to manually parse. Words for orientation are very common (s_ ing_ ed_ est_ er_ | _up _down _over _near _side _under)
2) Create rules for affix extraction so that the remaining word will be in the most commonly found state (extract ing_ from _writing_ should yield _write_ not _writ_)
3) Manually parse all contractions (words with ' (I'll / it's)) and remove all remaining words with apostrophes
4) Create fragments of words by sliding windows of size 2..9 over a word (observing a word fragment) and adding the word count to the tally for that fragment
5) Parse occuring more than 3000k times are moved to end dictionary
6) Parse occuring more than 100k times with 4 or less letters are moved to end dictionary
7) Parse with 3 or less letters are moved to end dictionary
8) Create a subset of single character affixes were manually identified but keep them in the affix list
9) Remove affixes that have no vowels (other than the single letter affixes) as the do not carry meaning
10) Filter out non-affix fragments (no _ indicator) that are length 2 or less
11) FIlter out non-affix fragments that do not have an affixed version (mip never occurs at the beginning or end of a word in the vocab and is not likely to be an affix)
12) Filter out non-affix fragments when the sum of their affixed versions occur far more often frequently
13) Apply affix counting algorithm on the affix list to amplify signal of true affixes
14) Iterate through fragments, filtering for fragments that only appear in 2 or less words, and remove from the affix list

Problems
Identifying the appropriate cutoff point for an affix series 's_', 'es_', 'ities_' all carry meaning and occur uniquely but 'ties_' does not
    Cannot cut off all branches after finding a fragment that carries no information
Counts are not reliable in all cases. Affixes that carry meaning could occur infrequently but consistently.
False extractions occur 't_' is an affix for _burnt_ but not for _beat_. How to discriminate?
False extractions can stop the proper extraction from occuring. extracting s_ _from viruses_ leaves _viruse_ which means es_ cannot be extracted
Nested ruled affixes. 'ities_' is two affixes. 'ies_' and 'ity_'
Uncertainty. is '_a' or '_ab' the affix for '_abbreviation_'
Count uncertainty. 'ng_' occurs more than 'ing_' but 'ing_' extracts leaving a coherent root word


Tasks
Eliminate nested affixes
Ruled replacement
Affix Tree

"""

import btk
import numpy as np
import pickle

from collections import Counter
from tqdm import tqdm

class AffixAnalyzer:

    def __init__(self, words, load=False):
        self.vowels = {'a', 'e', 'i', 'o', 'u', 'y'}
        self.singles = {'s_', 'd_', 'r_', 'n_', 't_', 'x_', 'y_', 'a_', 'i_', 'o_', '_a', '_o', '_e', '_i'}
        self.doubles = {'b', 'c', 'd', 'f', 'g', 'l', 'm', 'n', 'p', 'r', 's', 't'}
        self.cleared = Counter()
        self.frags = Counter()
        self.final = Counter()
        if load:
            self.load()
        else:
            self.wlst = Counter({f'_{x[1]}_': int(x[0]) for x in words[::-1]})
            self.prep_frags()
            self.clean_contractions()
            self.create_frags()
            self.clean_common()

    def wcnt_to_occnt(self, wcnt):
        return round(np.log2(wcnt), 3) ** 3

    def get_afxline(self, afx, counts=False):
        if counts:
            return [x for x in self.frags.most_common() if afx in x[0] or x[0] in afx]
        else:
            return [x[0] for x in self.frags.most_common() if afx in x[0] or x[0] in afx]

    def get_afxwords(self, afx, counts=False):
        if counts:
            return [x for x in self.wlst.most_common() if afx in x[0]]
        else:
            return {x[0] for x in self.wlst.most_common() if afx in x[0]}

    def get_afx(self, afx, counts=False):
        if counts:
            return [x for x in self.frags.most_common() if afx in x[0]]
        else:
            return {x[0] for x in self.frags.most_common() if afx in x[0]}

    def get_rep(self, afx, word):
        word = word.replace(afx, '_')
        if afx[0] == '_':
            if (word[1] == word[2] or word[1] == afx[-1]) and word[1] in self.doubles:
                word = f'_{word[2:]}'
        else:
            if (word[-2] == word[-3] or word[-2] == afx[0]) and word[-2] in self.doubles:
                word = f'{word[:-2]}_'
        if word in self.wlst or word in self.cleared or word in self.final:
            return True

    def seek_tree(self, keys):
        for x in keys:
            target = self.tree[x]
        return target

    def degrade(self, afx):
        if afx[0] == '_':
            return afx[1:]
        elif afx[-1] == '_':
            return afx[:-1]

    def clean_common(self):
        for x in self.wlst.most_common():
            if x[1] > 3000000:
                self.cleared[x[0]] = x[1]
            elif x[1] > 100000 and len(x[0]) < 6:
                self.cleared[x[0]] = x[1]
            elif x[1] < 100000:
                break
        for z in [y for y in self.wlst if len(y) < 6]:
            self.cleared[z] += self.wlst[z]
        for z in [x for x in self.frags if not any(y in x for y in self.vowels) and x not in self.singles]:
            self.frags.pop(z)
        for z in [y for y in self.frags if len(y) == 2 and y not in self.singles]:
            self.frags.pop(z)
        for z in {x for x in self.frags if '_' not in x and (f'{x}_' in self.frags or f'_{x}' in self.frags) and (f'_{x}_' in self.cleared or f'_{x}_' in self.final or f'_{x}_' in self.wlst)}:
            self.frags.pop(z)
        for z in {x for x in self.frags if '_' not in x and (self.frags[f'_{x}'] + self.frags[f'{x}_']) / self.frags[x] > 0.666}:
            self.frags.pop(z)
        for z in self.cleared:
            if z in self.wlst:
                self.wlst.pop(z)
        for _ in range(2):
            for word in self.frags:
                tmplen = min(len(word), 10)
                for n in range(2, tmplen+1):
                    for pos in range(tmplen-n+2):
                        if word[pos:pos+n] in self.frags:
                            self.frags[word[pos:pos+n]] += self.frags[word]
        solos = []
        for afx in self.frags:
            c = 0
            for w in self.wlst:
                if afx in w:
                    c += 1
                if c > 2:
                    break
            else:
                solos.append(afx)
        for x in solos:
            self.frags.pop(x)
        self.frags = Counter({x[0]: int(x[1]**(1/np.e)) for x in self.frags.most_common()})
        self.frags['_'] = 10000

    def clean_contractions(self):
        for x in ["_ain't_", "_can't_", "_won't_", "_shan't_"]:
            self.final["n't_"] += self.wlst[x]
        for x in ["_i'm_", "_can't_"]:
            self.wlst[f'{x[:-3]}_'] += self.wlst[x]
        for x in ["_ma'am_", "_ain't_", "_i'm_"]:
            self.final[x] += self.wlst[x]
        for x in ["_ain't_", "_can't_", "_won't_", "_shan't_", "_ma'am_", "_i'm_", "_van't_"]:
            self.wlst.pop(x)
        for x in [z for z in self.wlst if "'" in z if any(y in z for y in ["'s_", "'ll_", "'ve_", "n't_", "'re_", "'d_"])]:
            for efx in ["'s_", "'ll_", "'ve_", "n't_", "'re_", "'d_"]:
                if efx in x:
                    self.final[efx] += self.wlst[x]
                    self.wlst[x.replace(efx, '_')] += self.wlst[x]
                    self.wlst.pop(x)
        for x in [z for z in self.wlst if "'" in z]:
            self.wlst.pop(x)

    def prep_frags(self):
        with open(r'D:\dstore\nlp\w2v\directions', 'rb') as f:
            directions = pickle.load(f)
        self.frags['_a'] += self.wlst["_around_"]
        self.frags["_round"] += self.wlst["_around_"]
        self.frags["_o"] += self.wlst["_over_"]
        self.frags["_ver"] += self.wlst["_over_"]
        for x in directions[0]:
            self.frags[x[:-1]] += self.wlst[x]
        for x in directions[2]:
            self.frags[x[:-1]] += self.wlst[x]
            self.frags[x[1:]] += self.wlst[x]
        for x in directions[1]:
            self.frags[x[1:]] += self.wlst[x]
        for y in directions:
            for x in y:
                if len(x) < 5:
                    self.final[x] += self.wlst[x]
                    self.wlst.pop(x)
                else:
                    self.cleared[x] += self.wlst[x]
                    self.wlst.pop(x)

    def create_frags(self):
        for word in self.wlst:
            tmplen = min(len(word), 10)
            for n in range(2, tmplen-1):
                for pos in range(tmplen-n+1):
                    self.frags[word[pos:pos+n]] += self.wlst[word]
        for x in {y for y in self.frags if '_' in y and len(y) == 2}:
            if x not in self.singles:
                self.frags.pop(x)
        for x in [y for y in self.frags if '_' not in y]:
            if f'_{x}' not in self.frags and f'{x}_' not in self.frags:
                self.frags.pop(x)
        tmp = [x[0] for x in self.frags.most_common()]
        for i, x in enumerate(tmp):
            lim = 0
            while lim < 2048:
                try:
                    if x in tmp[i+1+lim]:
                        self.frags[x] -= self.frags[tmp[i+1+lim]]
                        break
                except IndexError:
                    break
                lim += 1
        self.frags = Counter({x[0]: x[1] for x in self.frags.most_common() if x[1] > 128})

    def sieve_level(self, inp):
        out = []
        i = min(len(x) for x in inp)
        while inp or i <= 8:
            active = {x for x in inp if len(x) == i}
            for x in active:
                tmp = {y for y in inp if x in y}
                for y in tmp:
                    inp.remove(y)
                out.append(x)
            i += 1
        return out

    def af_tree(self):
        longest = max({len(x) for x in self.frags if '_' in x})
        self.tree = {'_': {x: None for x in self.frags if '_' in x and len(x) == 2}}
        targets = {x for x in self.frags if any(y in x for y in self.tree['_'])}
        for x in self.tree['_']: targets.remove(x)
        tracker = ['_']
        
        while targets:
            pass

    def load(self):
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\words', 'rb') as f:
            self.wlst = pickle.load(f)
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\frags', 'rb') as f:
            self.frags = pickle.load(f)
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\cleared', 'rb') as f:
            self.cleared = pickle.load(f)
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\final', 'rb') as f:
            self.final = pickle.load(f)

    def save(self):
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\words', 'wb') as f:
            pickle.dump(self.wlst, f)
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\frags', 'wb') as f:
            pickle.dump(self.frags, f)
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\cleared', 'wb') as f:
            pickle.dump(self.cleared, f)
        with open(r'C:\Users\BBA\Coding\NLP\Embeddings\data\final', 'wb') as f:
            pickle.dump(self.final, f)

with open(r'D:\dstore\nlp\w2v\fwords', 'rt') as f:
    afa = AffixAnalyzer([x.strip().split() for x in f.readlines()], True)


In [ ]:
with open(r'D:\dstore\nlp\w2v\word_nests2', 'rb') as f:
    solos, multi, afx_cand = pickle.load(f)

with open(r'D:\dstore\nlp\w2v\word_nests2', 'wb') as f:
    pickle.dump((solos, multi), f)


In [ ]:
test = 'ated_'
keys = [test[i:] for i, x in enumerate(test)][::-1][1:-1]
tdct = {'_': {'d_': {'ed_': {'ted_': {'ated_': False}}}}}